In [ ]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 23.9 MB/s eta 0:00:00


In [ ]:
# https://docs.google.com/spreadsheets/d/1CAzsKw0GvI7la2iWbsViBGUTJILOjTYagwxvAvGtatg/edit?usp=sharing

import os
import json
import random
import re
from tqdm.auto import tqdm
import warnings

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import numpy as np
import torch

import category_encoders as ce
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score, accuracy_score
import lightgbm as lgb
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
import optuna

In [ ]:
def set_seed(seed):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    torch.use_deterministic_algorithms(False)
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

    print(f"Random seed set to {seed}")

In [ ]:
RANDOM_SEED = 42
set_seed(RANDOM_SEED)

warnings.filterwarnings("ignore", category=UserWarning)
optuna.logging.set_verbosity(optuna.logging.WARNING)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {DEVICE}")

Random seed set to 42
Device: cuda


In [ ]:
INPUT_ROOT = "/content/drive/MyDrive/DataFlow2026/data"
WORK_DIR = "/content/drive/MyDrive/DataFlow2026/working"

ACADEMIC_CSV = f"{INPUT_ROOT}/academic_records.csv"
ADMISSION_CSV = f"{INPUT_ROOT}/admission.csv"
TEST_CSV = f"{INPUT_ROOT}/test.csv"

SUBMISSION_CSV = f"{WORK_DIR}/submission.csv"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
admission_df = pd.read_csv(ADMISSION_CSV)
academic_df = pd.read_csv(ACADEMIC_CSV)

semester_order = [
    "HK1 2020-2021", "HK2 2020-2021",
    "HK1 2021-2022", "HK2 2021-2022",
    "HK1 2022-2023", "HK2 2022-2023",
    "HK1 2023-2024", "HK2 2023-2024",
    "HK1 2024-2025"
]
semester_mapping = {sem: i for i, sem in enumerate(semester_order)}

student_df = pd.merge(academic_df, admission_df, on="MA_SO_SV", how="left")
student_df["SEMESTER_INDEX"] = student_df["HOC_KY"].map(semester_mapping)

student_df = student_df.sort_values(
    by=["MA_SO_SV", "HOC_KY", "TC_DANGKY", "TC_HOANTHANH", "GPA"],
    ascending=[True, True, True, False, False]
)
student_df = student_df.drop_duplicates(subset=["MA_SO_SV", "HOC_KY"], keep="first")
student_df = student_df.sort_values(["MA_SO_SV", "SEMESTER_INDEX"]).reset_index(drop=True)

valid_semesters = ["HK2 2023-2024"]
train_df_raw = student_df[~student_df["HOC_KY"].isin(valid_semesters)].copy()
val_df_raw = student_df[student_df["HOC_KY"].isin(valid_semesters)].copy()

test_df_raw = pd.read_csv(TEST_CSV)
test_df_raw = pd.merge(test_df_raw, admission_df, on="MA_SO_SV", how="left")
test_df_raw["SEMESTER_INDEX"] = test_df_raw["HOC_KY"].map(semester_mapping)

print(f"Raw Train shape: {train_df_raw.shape}")
print(f"Raw Val shape: {val_df_raw.shape}")
print(f"Raw Test shape: {test_df_raw.shape}")

Raw Train shape: (90122, 12)
Raw Val shape: (15144, 12)
Raw Test shape: (16502, 9)


In [ ]:
EXAM_STATS_DETAILED = {
    2020: {
        "A00": {"mean": 21.4471, "std": 3.3425},
        "A01": {"mean": 20.0417, "std": 3.3084},
        "B00": {"mean": 20.3388, "std": 3.0812},
        "D01": {"mean": 18.1417, "std": 3.7811},
        "D07": {"mean": 20.0117, "std": 3.1333},
        "D24": {"mean": 22.8503, "std": 2.8891},
        "D29": {"mean": 22.3681, "std": 2.8403},
        "OTHER": {"mean": 19.5364, "std": 3.6661}
    },
    2021: {
        "A00": {"mean": 21.0262, "std": 3.1863},
        "A01": {"mean": 21.1029, "std": 3.4455},
        "B00": {"mean": 19.9892, "std": 3.0890},
        "D01": {"mean": 19.2666, "std": 4.1166},
        "D07": {"mean": 21.1338, "std": 3.2841},
        "D24": {"mean": 21.9896, "std": 3.0011},
        "D29": {"mean": 21.4177, "std": 3.0370},
        "OTHER": {"mean": 20.2071, "std": 3.7104}
    },
    2022: {
        "A00": {"mean": 21.0955, "std": 3.2378},
        "A01": {"mean": 20.2909, "std": 3.3396},
        "B00": {"mean": 19.4039, "std": 3.1555},
        "D01": {"mean": 18.4381, "std": 3.8846},
        "D07": {"mean": 20.2397, "std": 3.2064},
        "OTHER": {"mean": 19.5196, "std": 3.6548}
    },
    2023: {
        "A00": {"mean": 20.7745, "std": 3.0941},
        "A01": {"mean": 20.2743, "std": 3.3399},
        "B00": {"mean": 20.6047, "std": 2.7763},
        "D01": {"mean": 18.8891, "std": 3.8137},
        "D07": {"mean": 20.4216, "std": 3.0619},
        "D24": {"mean": 20.8364, "std": 3.3689},
        "D29": {"mean": 20.2285, "std": 3.5744},
        "OTHER": {"mean": 19.8594, "std": 3.4893}
    },
    2024: {
        "A00": {"mean": 20.9046, "std": 3.3804},
        "A01": {"mean": 20.4724, "std": 3.3509},
        "B00": {"mean": 20.5311, "std": 2.9818},
        "D01": {"mean": 19.4939, "std": 3.6232},
        "D07": {"mean": 20.4510, "std": 3.1120},
        "D24": {"mean": 21.8162, "std": 3.2930},
        "D29": {"mean": 20.9934, "std": 3.5598},
        "OTHER": {"mean": 20.1512, "std": 3.4281}
    },
}

In [ ]:
def get_features(input_df):
    df = input_df.copy()

    target_keys = set(zip(df["MA_SO_SV"], df["HOC_KY"]))
    student_filtered = student_df[
        ~student_df.set_index(["MA_SO_SV", "HOC_KY"]).index.isin(target_keys)
    ].copy()

    df = pd.concat([student_filtered, df], axis=0, ignore_index=True)
    df = df.sort_values(["MA_SO_SV", "SEMESTER_INDEX"])

    df["PTXT"] = df["PTXT"].replace({
        "5": "100",
        "3": "200",
        "1": "303"
    })

    def calculate_z_score(row):
        year = row["NAM_TUYENSINH"]
        score = row["DIEM_TRUNGTUYEN"]
        block = str(row["TOHOP_XT"]).upper().strip()

        if year not in EXAM_STATS_DETAILED:
            return np.nan

        year_stats = EXAM_STATS_DETAILED[year]
        if block in year_stats:
            stats = year_stats[block]
        else:
            stats = year_stats["OTHER"]

        mean_val = stats["mean"]
        std_val = stats["std"]

        if std_val < 0.1:
            std_val = 1.0

        return (score - mean_val) / std_val

    df["Z_SCORE"] = df.apply(calculate_z_score, axis=1)

    if "TC_HOANTHANH" in df.columns:
        df["FAIL_CREDITS"] = df["TC_DANGKY"] - df["TC_HOANTHANH"]
    else:
        df["FAIL_CREDITS"] = 0

    df["SCORE_GAP"] = df["DIEM_TRUNGTUYEN"] - df["DIEM_CHUAN"]
    df["GAP_RATIO"] = df["SCORE_GAP"] / (df["DIEM_CHUAN"] + 1.0)

    df["ENTRY_RANK"] = df.groupby("NAM_TUYENSINH")["DIEM_TRUNGTUYEN"].transform(
        lambda x: x.rank(pct=True, method='average')
    )
    df["BENCHMARK_TIER"] = df.groupby("NAM_TUYENSINH")["DIEM_CHUAN"].transform(
        lambda x: x.rank(pct=True)
    )

    grouped = df.groupby("MA_SO_SV")
    df["LAST_GPA"] = grouped["GPA"].shift(1)
    df["LAST_FAIL"] = grouped["FAIL_CREDITS"].shift(1)
    df["LAST_PASSED"] = grouped["TC_HOANTHANH"].shift(1)
    df["LAST_DANGKY"] = grouped["TC_DANGKY"].shift(1)
    df["LAST_PASS_RATIO"] = df["LAST_PASSED"] / df["LAST_DANGKY"]

    df["HIST_AVG_GPA"] = grouped["GPA"].transform(lambda x: x.shift(1).expanding().mean())
    df["TOTAL_EARNED"] = grouped["TC_HOANTHANH"].transform(lambda x: x.shift(1).fillna(0).cumsum())
    df["HIST_MAX_PASSED"] = grouped["TC_HOANTHANH"].transform(lambda x: x.shift(1).expanding().max())
    df["HIST_MAX_GPA"] = grouped["GPA"].transform(lambda x: x.shift(1).expanding().max())
    df["HIST_STD_GPA"] = grouped["GPA"].transform(lambda x: x.shift(1).expanding().std())
    df["OVERLOAD_VS_MAX"] = df["TC_DANGKY"] - df["HIST_MAX_PASSED"]

    window2 = grouped.rolling(window=2, min_periods=1, closed='left')
    df["R2_AVG_GPA"] = window2["GPA"].mean().reset_index(0, drop=True)
    df["R2_SUM_FAIL"] = window2["FAIL_CREDITS"].sum().reset_index(0, drop=True)
    df["R2_AVG_PASSED"] = window2["TC_HOANTHANH"].mean().reset_index(0, drop=True)
    df["R2_SUM_DANGKY"] = window2["TC_DANGKY"].sum().reset_index(0, drop=True)
    df["R2_SUM_PASSED"] = window2["TC_HOANTHANH"].sum().reset_index(0, drop=True)
    df["R2_PASS_RATE"] = df["R2_SUM_PASSED"] / df["R2_SUM_DANGKY"]
    df["PRESSURE_VS_R2"] = df["TC_DANGKY"] / df["R2_AVG_PASSED"]
    df["GPA_TREND_R2"] = df["R2_AVG_GPA"] - df["HIST_AVG_GPA"]
    df["FAIL_TREND_R2"] = df["LAST_FAIL"] - (df["R2_SUM_FAIL"] / 2)

    window3 = grouped.rolling(window=3, min_periods=1, closed='left')
    df["R3_AVG_GPA"] = window3["GPA"].mean().reset_index(0, drop=True)
    df["R3_SUM_FAIL"] = window3["FAIL_CREDITS"].sum().reset_index(0, drop=True)
    df["R3_AVG_PASSED"] = window3["TC_HOANTHANH"].mean().reset_index(0, drop=True)
    df["PRESSURE_VS_R3"] = df["TC_DANGKY"] / df["R3_AVG_PASSED"]
    df["OVERLOAD_R3"] = df["TC_DANGKY"] - df["R3_AVG_PASSED"]

    def parse_hoc_ky(hk_str):
        hk_str = str(hk_str)
        year_match = re.search(r"(\d{4})", hk_str)
        year = int(year_match.group(1)) if year_match else 2024
        return year

    df["YEAR_START"] = df["HOC_KY"].apply(parse_hoc_ky)
    df["SV_NAM_THU"] = df["YEAR_START"] - df["NAM_TUYENSINH"] + 1
    df.loc[df["SV_NAM_THU"] < 1, "SV_NAM_THU"] = 1

    fill_0 = [
        "LAST_FAIL", "R2_SUM_FAIL", "R3_SUM_FAIL", "FAIL_TREND_R2", "OVERLOAD_R3",
        "TOTAL_EARNED", "OVERLOAD_VS_MAX", "HIST_STD_GPA", "GPA_TREND_R2"
    ]
    df[fill_0] = df[fill_0].fillna(0.0)

    fill_1 = ["LAST_PASS_RATIO", "R2_PASS_RATE", "PRESSURE_VS_R2", "PRESSURE_VS_R3"]
    df[fill_1] = df[fill_1].fillna(1.0)

    fill_mean_gpa = ["LAST_GPA", "R2_AVG_GPA", "R3_AVG_GPA", "HIST_AVG_GPA", "HIST_MAX_GPA"]
    df[fill_mean_gpa] = df[fill_mean_gpa].fillna(academic_df["GPA"].mean())

    fill_15 = ["LAST_PASSED", "R2_AVG_PASSED", "R3_AVG_PASSED", "HIST_MAX_PASSED"]
    df[fill_15] = df[fill_15].fillna(15.0)

    df = df.replace([np.inf, -np.inf], np.nan).fillna(0.0)

    final_df = pd.merge(
        input_df[["MA_SO_SV", "HOC_KY"]],
        df,
        on=["MA_SO_SV", "HOC_KY"],
        how="left"
    )

    return final_df

train_final = get_features(train_df_raw)
val_final = get_features(val_df_raw)
test_final = get_features(test_df_raw)

In [ ]:
print(train_final.columns)
print(val_final.columns)
print(test_final.columns)

Index(['MA_SO_SV', 'HOC_KY', 'CPA', 'GPA', 'TC_DANGKY', 'TC_HOANTHANH',
       'NAM_TUYENSINH', 'PTXT', 'TOHOP_XT', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN',
       'SEMESTER_INDEX', 'Z_SCORE', 'FAIL_CREDITS', 'SCORE_GAP', 'GAP_RATIO',
       'ENTRY_RANK', 'BENCHMARK_TIER', 'LAST_GPA', 'LAST_FAIL', 'LAST_PASSED',
       'LAST_DANGKY', 'LAST_PASS_RATIO', 'HIST_AVG_GPA', 'TOTAL_EARNED',
       'HIST_MAX_PASSED', 'HIST_MAX_GPA', 'HIST_STD_GPA', 'OVERLOAD_VS_MAX',
       'R2_AVG_GPA', 'R2_SUM_FAIL', 'R2_AVG_PASSED', 'R2_SUM_DANGKY',
       'R2_SUM_PASSED', 'R2_PASS_RATE', 'PRESSURE_VS_R2', 'GPA_TREND_R2',
       'FAIL_TREND_R2', 'R3_AVG_GPA', 'R3_SUM_FAIL', 'R3_AVG_PASSED',
       'PRESSURE_VS_R3', 'OVERLOAD_R3', 'YEAR_START', 'SV_NAM_THU'],
      dtype='object')
Index(['MA_SO_SV', 'HOC_KY', 'CPA', 'GPA', 'TC_DANGKY', 'TC_HOANTHANH',
       'NAM_TUYENSINH', 'PTXT', 'TOHOP_XT', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN',
       'SEMESTER_INDEX', 'Z_SCORE', 'FAIL_CREDITS', 'SCORE_GAP', 'GAP_RATIO',
       'ENTR

In [ ]:
target = "FAIL_CREDITS"
categorical_cols = ["PTXT", "TOHOP_XT"]

cbe = ce.CatBoostEncoder(
    cols=categorical_cols,
    handle_missing="return_nan"
)

train_final[categorical_cols] = cbe.fit_transform(
    train_final[categorical_cols],
    train_final[target]
)

val_final[categorical_cols] = cbe.transform(val_final[categorical_cols])
test_final[categorical_cols] = cbe.transform(test_final[categorical_cols])

print(f"Final Train shape: {train_final.shape}")
print(f"Final Val shape: {val_final.shape}")
print(f"Final Test shape: {test_final.shape}")

Final Train shape: (90122, 45)
Final Val shape: (15144, 45)
Final Test shape: (16502, 45)


In [ ]:
def split_by_year(df):
    df_fresh = df[df["SV_NAM_THU"] == 1].copy()
    df_senior = df[df["SV_NAM_THU"] > 1].copy()

    return df_fresh, df_senior

train_fresh, train_senior = split_by_year(train_final)
val_fresh, val_senior = split_by_year(val_final)
test_fresh, test_senior = split_by_year(test_final)

In [ ]:
feats_senior = [
    "TC_DANGKY", "SEMESTER_INDEX", "SV_NAM_THU",

    "LAST_GPA", "LAST_FAIL", "LAST_PASS_RATIO",

    "R2_AVG_GPA", "R2_SUM_FAIL", "R2_PASS_RATE",
    "R3_AVG_GPA", "R3_SUM_FAIL",

    "PRESSURE_VS_R2", "PRESSURE_VS_R3", "OVERLOAD_R3",
    "FAIL_TREND_R2", "GPA_TREND_R2",

    "TOTAL_EARNED", "HIST_AVG_GPA",
    "HIST_MAX_PASSED",
    "HIST_MAX_GPA",
    "HIST_STD_GPA",
    "OVERLOAD_VS_MAX"
]

feats_fresh = [
    "TC_DANGKY", "SEMESTER_INDEX",
    "PTXT", "TOHOP_XT",

    "DIEM_TRUNGTUYEN", "DIEM_CHUAN",
    "SCORE_GAP", "ENTRY_RANK", "BENCHMARK_TIER",
    "Z_SCORE", "GAP_RATIO",

    "LAST_GPA", "LAST_FAIL", "LAST_PASS_RATIO",
    "PRESSURE_VS_R2"
]

meta_cols = ["MA_SO_SV", "HOC_KY", "FAIL_CREDITS"]

def filter_cols(df, features):
    desired_cols = set(features + meta_cols)
    existing_cols = [c for c in df.columns if c in desired_cols]
    return df[existing_cols].copy()

train_fresh = filter_cols(train_fresh, feats_fresh)
val_fresh = filter_cols(val_fresh, feats_fresh)
test_fresh = filter_cols(test_fresh, feats_fresh)

train_senior = filter_cols(train_senior, feats_senior)
val_senior = filter_cols(val_senior, feats_senior)
test_senior = filter_cols(test_senior, feats_senior)

full_train_fresh = pd.concat([train_fresh, val_fresh], axis=0, ignore_index=True)
full_train_senior = pd.concat([train_senior, val_senior], axis=0, ignore_index=True)

In [ ]:
print(f"Train Fresher: {train_fresh.shape} | Train Senior: {train_senior.shape}")
print(f"Val Fresher: {val_fresh.shape} | Val Senior: {val_senior.shape}")
print(f"Test Fresher: {test_fresh.shape} | Test Senior: {test_senior.shape}")

Train Fresher: (24996, 18) | Train Senior: (65126, 25)
Val Fresher: (3504, 18) | Val Senior: (11640, 25)
Test Fresher: (4326, 18) | Test Senior: (12176, 25)


In [ ]:
FIXED_PARAMS = {
    "objective": "tweedie",
    "metric": "rmse",
    "boosting_type": "gbdt",
    "random_state": RANDOM_SEED,
    "n_estimators": 4000,
    "device": DEVICE.type,
    "verbosity": -1,
    "boost_from_average": True
}

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import optuna
import numpy as np
from tqdm.auto import tqdm
import warnings

warnings.filterwarnings('ignore')

def optimize_random_forest(train_df, val_df, feats, target_col, n_trial, optim_type=""):

    X_train = train_df[feats]
    y_train = train_df[target_col]
    X_val = val_df[feats]
    y_val = val_df[target_col]

    def objective_rf(trial):
        bootstrap = trial.suggest_categorical("bootstrap", [True, False])

        if bootstrap:
            max_samples = trial.suggest_float("max_samples", 0.5, 0.95)
        else:
            max_samples = None
        max_features_option = trial.suggest_categorical("max_features_opt", ["sqrt", "log2", "custom_float"])

        if max_features_option == "custom_float":
            max_features = trial.suggest_float("max_features_val", 0.1, 0.9)
        else:
            max_features = max_features_option

        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 500),
            "max_depth": trial.suggest_int("max_depth", 10, 50),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 15),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),

            "max_features": max_features,
            "bootstrap": bootstrap,
            "max_samples": max_samples,

            "random_state": 42,
            "n_jobs": -1
        }

        model = RandomForestRegressor(**params)

        model.fit(X_train, y_train)

        preds = model.predict(X_val)

        preds = np.maximum(preds, 0)
        if "TC_DANGKY" in val_df.columns:
            limit = val_df["TC_DANGKY"].values
            preds = np.minimum(preds, limit)

        return np.sqrt(mean_squared_error(y_val, preds))

    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))

    print(f" Tuning Advanced Random Forest {optim_type} with {n_trial} trials...")
    with tqdm(total=n_trial, desc=f"RF Adv {optim_type}") as pbar:
        def tqdm_callback(study, trial):
            pbar.update(1)
            if study.best_value:
                pbar.set_postfix({"Best RMSE": f"{study.best_value:.4f}"})

        study.optimize(objective_rf, n_trials=n_trial, callbacks=[tqdm_callback])

    print(f"\n Best RMSE: {study.best_value:.4f}")
    return study.best_params, study.best_value

In [ ]:
N_TRIALS = 100

best_fresh_params, best_fresh_rmse = optimize_random_forest(
    train_fresh, val_fresh, feats_fresh,"FAIL_CREDITS", N_TRIALS, "Fresher"
)

best_senior_params, best_senior_rmse = optimize_random_forest(
    train_senior, val_senior, feats_senior, "FAIL_CREDITS", N_TRIALS, "Senior"
)

 Tuning Advanced Random Forest Fresher with 100 trials...


RF Adv Fresher:   0%|          | 0/100 [00:00<?, ?it/s]


 Best RMSE: 3.9706
 Tuning Advanced Random Forest Senior with 100 trials...


RF Adv Senior:   0%|          | 0/100 [00:00<?, ?it/s]


 Best RMSE: 3.6808


In [ ]:
print(f"Best Fresher RMSE: {best_fresh_rmse:.4f}")
print(f"Best Senior RMSE: {best_senior_rmse:.4f}")

print(f"Best Fresher Params: {best_fresh_params}")
print(f"Best Senior Params: {best_senior_params}")

Best Fresher RMSE: 3.9706
Best Senior RMSE: 3.6808
Best Fresher Params: {'bootstrap': True, 'max_samples': 0.6440881676799254, 'max_features_opt': 'custom_float', 'max_features_val': 0.49958512791451803, 'n_estimators': 237, 'max_depth': 37, 'min_samples_split': 7, 'min_samples_leaf': 10}
Best Senior Params: {'bootstrap': True, 'max_samples': 0.5658575838197372, 'max_features_opt': 'log2', 'n_estimators': 414, 'max_depth': 10, 'min_samples_split': 8, 'min_samples_leaf': 2}


In [ ]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np

def train_random_forest(params, train_df, val_df, feats, model_type, target_col="FAIL_CREDITS"):
    print(f"Training {model_type} Model (Random Forest)...")

    rf_params = params.copy()

    if "max_features_opt" in rf_params:
        option = rf_params.pop("max_features_opt")

        if option == "custom_float":
            val = rf_params.pop("max_features_val", 0.5)
            rf_params["max_features"] = val
        else:
            rf_params["max_features"] = option
            if "max_features_val" in rf_params:
                del rf_params["max_features_val"]

    rf_params["n_jobs"] = -1
    if "random_state" not in rf_params:
        rf_params["random_state"] = 42

    try:
        model = RandomForestRegressor(**rf_params)
    except TypeError as e:
        print(" Lỗi tham số:", e)
        print("Params hiện tại:", rf_params)
        raise e

    model.fit(
        train_df[feats],
        train_df[target_col]
    )
    preds = model.predict(val_df[feats])
    if "TC_DANGKY" in val_df.columns:
        limit = val_df["TC_DANGKY"].values
        preds = np.clip(preds, 0, limit)
    else:
        preds = np.maximum(preds, 0)

    return model, preds

target_variable = "FAIL_CREDITS"

_, preds_fresh = train_random_forest(
    best_fresh_params,
    train_fresh,
    val_fresh,
    feats_fresh,
    "Fresher",
    target_col=target_variable
)

_, preds_senior = train_random_forest(
    best_senior_params,
    train_senior,
    val_senior,
    feats_senior,
    "Senior",
    target_col=target_variable
)

Training Fresher Model (Random Forest)...
Training Senior Model (Random Forest)...


In [ ]:
val_targets = np.concatenate([val_fresh[target].values, val_senior[target].values])
val_preds = np.concatenate([preds_fresh, preds_senior])

def calculate_wmape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    if np.sum(y_true) == 0:
        return 0
    return np.sum(np.abs(y_true - y_pred)) / np.sum(y_true)

rmse = np.sqrt(mean_squared_error(val_targets, val_preds))
mse = mean_squared_error(val_targets, val_preds)
r2 = r2_score(val_targets, val_preds)
wmape = calculate_wmape(val_targets, val_preds)

print(f"RMSE : {rmse:.4f}")
print(f"MSE : {mse:.4f}")
print(f"R^2 : {r2:.4f}")
print(f"wMAPE : {wmape:.4f}")

RMSE : 3.7498
MSE : 14.0611
R^2 : 0.5038
wMAPE : 0.6400


In [ ]:
import joblib
import os
import numpy as np
from sklearn.ensemble import RandomForestRegressor

def test_random_forest(params, full_train_df, test_df, feats, model_type, target_col="FAIL_CREDITS"):
    print(f"Testing {model_type} Model (Random Forest)...")

    final_params = params.copy()

    if "max_features_opt" in final_params:
        option = final_params.pop("max_features_opt")

        if option == "custom_float":
            val = final_params.pop("max_features_val", 0.5)
            final_params["max_features"] = val
        else:
            final_params["max_features"] = option
            if "max_features_val" in final_params:
                del final_params["max_features_val"]

    final_params["n_jobs"] = -1
    final_params["random_state"] = 42

    try:
        model = RandomForestRegressor(**final_params)
    except TypeError as e:
        print(f" Lỗi tham số khi khởi tạo {model_type}:", e)
        print("Params:", final_params)
        raise e

    print(f"-> Fitting on full data ({len(full_train_df)} rows)...")
    model.fit(
        full_train_df[feats],
        full_train_df[target_col]
    )

    # 4. Lưu model
    if not os.path.exists(WORK_DIR):
        os.makedirs(WORK_DIR)
    joblib.dump(model, f"{WORK_DIR}/rf_{model_type.lower()}.pkl")

    # 5. Dự báo
    preds = model.predict(test_df[feats])

    # 6. Hậu xử lý
    if "TC_DANGKY" in test_df.columns:
        limit = test_df["TC_DANGKY"].values
        preds = np.clip(preds, 0, limit)
        preds = limit - preds
    else:
        preds = np.maximum(preds, 0)

    return preds


In [ ]:
preds_fresh = test_random_forest(
    best_fresh_params,
    full_train_fresh,
    test_fresh,
    feats_fresh,
    "Fresher"
)

# Dự báo Senior
preds_senior = test_random_forest(
    best_senior_params,
    full_train_senior,
    test_senior,
    feats_senior,
    "Senior"
)

# Kiểm tra vài mẫu kết quả
print("\nSample Fresh Preds:", preds_fresh[:5])
print("Sample Senior Preds:", preds_senior[:5])

Testing Fresher Model (Random Forest)...
-> Fitting on full data (28500 rows)...
Testing Senior Model (Random Forest)...
-> Fitting on full data (76766 rows)...

Sample Fresh Preds: [18.74868885 18.81562141 16.04584935 17.5566412  16.53192652]
Sample Senior Preds: [ 1.62077485  2.37280135  3.91075286  0.         14.60972997]


In [ ]:
sub_fresh_df = pd.DataFrame({
    "MA_SO_SV": test_fresh["MA_SO_SV"],
    "PRED_TC_HOANTHANH": preds_fresh
})

sub_senior_df = pd.DataFrame({
    "MA_SO_SV": test_senior["MA_SO_SV"],
    "PRED_TC_HOANTHANH": preds_senior
})

preds_df = pd.concat([sub_fresh_df, sub_senior_df], axis=0)

submission = pd.merge(
    test_df_raw[["MA_SO_SV"]],
    preds_df,
    on=["MA_SO_SV"],
    how="left"
)

submission.to_csv(SUBMISSION_CSV, index=False)

In [ ]:
print(submission)

           MA_SO_SV  PRED_TC_HOANTHANH
0      481436e2064d           1.620775
1      6c8a97d22131           2.372801
2      e87f62beabbb           3.910753
3      438aff5ef524           0.000000
4      ad172a9b0722          14.609730
...             ...                ...
16497  9e803a0d26f0          39.892805
16498  dbc819721795          49.195782
16499  9e1c8deafb70          41.887716
16500  ffecfc70f83a          44.824047
16501  dc7b37953745          50.147845

[16502 rows x 2 columns]
